## Import libraries and dataset

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('adult.csv')
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


## Preprocessing

In [3]:
df.drop('fnlwgt',axis=1,inplace=True)

In [4]:
df.replace('?',np.nan,inplace=True)

### Handling Missing Values

In [5]:
df.isnull().sum()

age                  0
workclass         1836
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
income               0
dtype: int64

In [6]:
df['workclass'].fillna('unspecified workclass',inplace=True)
df['occupation'].fillna('unscpecified occupation',inplace=True)
df['native.country'].fillna('unspecified country',inplace=True)

### Column Transformer

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import category_encoders as ce

In [16]:
transformer = ColumnTransformer([
    ('onehot',OneHotEncoder(drop='first'),['relationship','race','sex']),
    ('binary',ce.BinaryEncoder(),['workclass','marital.status','occupation','native.country'])
],remainder='passthrough')

### Split Data

In [17]:
x = df.drop(['income','education'],axis=1)
y = df['income']

In [18]:
x_train,x_test,y_train,y_test = train_test_split(x,y,stratify=y,test_size=0.3,random_state=10)

### Transform Data

In [21]:
x_train_preprocessed = pd.DataFrame(transformer.fit_transform(x_train))
x_test_preprocessed = pd.DataFrame(transformer.transform(x_test))

C:\Users\Brian\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [28]:
features = list(transformer.transformers_[0][1].get_feature_names())+\
            transformer.transformers_[1][1].get_feature_names()+\
            ['age','education.num','capital.gain','capital.loss','hours.per.week']

In [29]:
x_train_preprocessed.columns=features
x_test_preprocessed.columns=features

## Build Model

In [39]:
tree = DecisionTreeClassifier(criterion='entropy',max_depth=5)
tree.fit(x_train_preprocessed,y_train)

y_pred = tree.predict(x_test_preprocessed)

print('Accuracy Score =',accuracy_score(y_test,y_pred))

Accuracy Score = 0.8447128672330843
